In [ ]:
import xml.etree.ElementTree as ET
import networkx as nx
import os
from math import sqrt
import matplotlib.pyplot as plt
from collections import deque, Counter

# Function to parse XPDL and order transitions correctly
def parse_xpdl(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    ns = {'xpdl': 'http://www.wfmc.org/2009/XPDL2.2'}
    
    # Lists
    pools = []
    lanes = []
    activities = []
    data_objects = []
    transitions = []  # Now using a list with manual duplicate check
    associations = []
    message_flows = []
    activity_id_to_name = {}  # Mapping from activity ID to name
    id_to_activity = {}  # Store activity details for reference
    
    # Parse Pools
    for pool in root.findall('.//xpdl:Pool', ns):
        pool_name = pool.get('Name')
        if pool_name:
            pools.append({'name': pool_name})
    
    # Parse Lanes
    for lane in root.findall('.//xpdl:Lane', ns):
        lane_name = lane.get('Name')
        if lane_name:
            lanes.append({'name': lane_name})
    
    # Parse Activities
    for activity in root.findall('.//xpdl:Activity', ns):
        activity_id = activity.get('Id')
        activity_name = activity.get('Name')
        event = activity.find('.//xpdl:Event', ns)
        route = activity.find('.//xpdl:Route', ns)
        gateway_type = route.get('GatewayType') if route is not None else 'None'
        is_start_event = False
        is_end_event = False
        
        if event is not None:
            if event.find('xpdl:StartEvent', ns) is not None:
                activity_name = 'StartEvent'
                is_start_event = True
            elif event.find('xpdl:EndEvent', ns) is not None:
                activity_name = 'EndEvent'
                is_end_event = True

        if activity_id:
            activity_detail = {
                'id': activity_id,
                'name': activity_name,
                'type': gateway_type,
                'is_start_event': is_start_event,
                'is_end_event': is_end_event,
                'from': [],
                'to': [],
                'data_objects': []  # Initialize an empty list for attached data objects
            }
            id_to_activity[activity_id] = activity_detail
            activity_id_to_name[activity_id] = activity_name if activity_name else activity_id

    # Parse DataObjects and assign them to the corresponding Activity
    for data_object in root.findall('.//xpdl:DataObject', ns):
        data_id = data_object.get('Id')
        data_name = data_object.get('Name')
        if data_id and data_name:
            for activity in id_to_activity.values():
                activity['data_objects'].append({'id': data_id, 'name': data_name, 'type': 'DataObject'})

    # Parse DataStores and assign them to the corresponding Activity
    for data_store in root.findall('.//xpdl:DataStore', ns):
        data_id = data_store.get('Id')
        data_name = data_store.get('Name')
        if data_id and data_name:
            for activity in id_to_activity.values():
                activity['data_objects'].append({'id': data_id, 'name': data_name, 'type': 'DataStore'})

            
    # Parse Transitions and create a transition map
    transition_map = {}  # Map each source to its target(s)
    reverse_transition_map = {}  # Map each target to its source(s)
    for transition in root.findall('.//xpdl:Transition', ns):
        source = transition.get('From')
        target = transition.get('To')
        if source and target:
            if source not in transition_map:
                transition_map[source] = []
            transition_map[source].append(target)

            if target not in reverse_transition_map:
                reverse_transition_map[target] = []
            reverse_transition_map[target].append(source)

            # Update `from` and `to` references
            if source in id_to_activity:
                id_to_activity[source]['to'].append(target)
            if target in id_to_activity:
                id_to_activity[target]['from'].append(source)
           
    # Add Associations to Transitions
    for association in root.findall('.//xpdl:Association', ns):
        source = association.get('Source')
        target = association.get('Target')
        if source and target:
            associations.append({'from': activity_id_to_name.get(source, source), 'to': activity_id_to_name.get(target, target)})

    # Add Message Flows to Transitions
    for message_flow in root.findall('.//xpdl:MessageFlow', ns):
        source = message_flow.get('Source')
        target = message_flow.get('Target')
        if source and target:
            message_flows.append({'from': activity_id_to_name.get(source, source), 'to': activity_id_to_name.get(target, target)})
                
    # Find the StartEvent node
    start_node = next((act for act in id_to_activity.values() if act['is_start_event']), None)
    if not start_node:
        raise ValueError("No StartEvent found in the XPDL.")

    # Track visited nodes
    visited = set()
    ordered_transitions = []  # Store transitions in order

    # Traverse the transitions in correct order using BFS
    queue = deque([start_node['id']])
    while queue:
        current_id = queue.popleft()
        if current_id in visited:
            continue
        visited.add(current_id)

        if current_id in transition_map:
            for target_id in transition_map[current_id]:
                transition_entry = {
                    'from': activity_id_to_name.get(current_id, current_id),
                    'to': activity_id_to_name.get(target_id, target_id)
                }

                # **Manually check for duplicates before adding**
                if transition_entry not in ordered_transitions:
                    ordered_transitions.append(transition_entry)
                queue.append(target_id)

    # Detect missing transitions
    missing_transitions = []
    for activity in id_to_activity.keys():
        if activity not in visited:
            missing_transitions.append(activity)

    # **Step 1: Reconstruct missing transitions using XPDL `from` and `to`**
    if missing_transitions:
        print("\n⚠️ Missing transitions detected! Attempting to reconstruct...")
        for missing_id in missing_transitions:
            missing_activity = id_to_activity[missing_id]
            print(f"  - {missing_activity['name']} ({missing_id}) is missing connections.")

            # Find the correct `from` node from the original XPDL transition list
            if missing_id in reverse_transition_map:
                correct_from = reverse_transition_map[missing_id][0]  # Take the first known connection
                correct_from_name = activity_id_to_name.get(correct_from, correct_from)

                transition_entry = {
                    'from': correct_from_name,
                    'to': activity_id_to_name.get(missing_id, missing_id)
                }
                if transition_entry not in ordered_transitions:  # Prevent duplicates
                    ordered_transitions.append(transition_entry)
                    print(f"  ✅ Reconstructed missing transition: {correct_from_name} → {missing_activity['name']}")

            if missing_id in transition_map:
                correct_to = transition_map[missing_id][0]  # Take the first known connection
                correct_to_name = activity_id_to_name.get(correct_to, correct_to)

                transition_entry = {
                    'from': activity_id_to_name.get(missing_id, missing_id),
                    'to': correct_to_name
                }
                if transition_entry not in ordered_transitions:  # Prevent duplicates
                    ordered_transitions.append(transition_entry)
                    print(f"  ✅ Reconstructed missing transition: {missing_activity['name']} → {correct_to_name}")

    # **Step 3: Append Associations & Message Flows in Order**
    ordered_transitions.extend(associations)
    ordered_transitions.extend(message_flows)
                    
    print("Final Transitions:", file_path, ordered_transitions)

    return {
        'pools': pools,
        'lanes': lanes,
        'activities': list(id_to_activity.values()),
        'data_objects': data_objects,
        'transitions': ordered_transitions,  # Use ordered transitions
        'associations': associations,
        'message_flows': message_flows
    }


def convert_to_interstructural_with_order(parsed_data):
    """
    Convert parsed XPDL data into a graph preserving the original process flow.
    Activities and transitions are added in the correct order.
    """
    G = nx.DiGraph()

    # Map activity IDs to their details for quick access
    activity_map = {activity['id']: activity for activity in parsed_data['activities']}

    # Step 1: Add all activities first but connect them through transitions
    print("parsed_data['transitions']", parsed_data['transitions'])
    for transition in parsed_data['transitions']:
        source_id = transition['from']
        target_id = transition['to']

        # Ensure the source and target activities exist
        source_activity = activity_map.get(source_id, {'name': source_id, 'type': 'Unknown'})
        target_activity = activity_map.get(target_id, {'name': target_id, 'type': 'Unknown'})
        print("source_id->", source_id, "|||", source_activity)
        print("target_id->", target_id, "|||", target_activity)

        if not source_activity['name']:
            source_activity['name'] = str(source_activity["type"]) + "_" + source_activity["id"]
            
        if not target_activity['name']:
            target_activity['name'] = str(target_activity["type"]) + "_" + target_activity["id"]
        
        # Add source activity if not already added
        if not G.has_node(source_activity['name']):
            G.add_node(source_activity['name'], label=source_activity['name'], type=source_activity.get('type', 'Activity'))
            
        # Add transition node
        transition_node = f"Transition_{source_activity['name']}_to_{target_activity['name']}"
        G.add_node(transition_node, label=transition_node, type="Transition")    
        
        # Add target activity if not already added
        if not G.has_node(target_activity['name']):
            G.add_node(target_activity['name'], label=target_activity['name'], type=target_activity.get('type', 'Activity'))

        # Connect source -> transition -> target
        G.add_edge(source_activity['name'], transition_node)  # Source to Transition
        G.add_edge(transition_node, target_activity['name'])  # Transition to Target

    return G

def text_to_vector(text):
    """
    Converts a string into a frequency vector.
    """
    words = text.lower().split()
    return Counter(words)

def cosine_similarity(text1, text2):
    """
    Calculates cosine similarity between two text strings.
    """
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    # print(vector1, vector2)

    intersection = set(vector1.keys()) & set(vector2.keys())
    dot_product = sum([vector1[word] * vector2[word] for word in intersection])

    magnitude1 = sqrt(sum([count ** 2 for count in vector1.values()]))
    magnitude2 = sqrt(sum([count ** 2 for count in vector2.values()]))

    if not magnitude1 or not magnitude2:
        return 0.0

    return dot_product / (magnitude1 * magnitude2)

def calculate_semantic_similarity(target_node, other_node, component):
    """
    Calculates semantic similarity between two nodes using cosine similarity.
    Missing sub-components are skipped, and the result is normalized by the total weight.
    """

    # Adjusted weights with 'label' and normalized to sum to 1
    sub_components_weights = {
        'pools': (['label'], [1.0]),
        'lanes': (['label'], [1.0]),
        'activities': (['label', 'type'], [0.5, 0.5]),
        'data_objects': (['label', 'type'], [0.5, 0.5]),
        'transitions': (['type'], [1.0]),  # Only check 'type' for transitions
        'associations': (['label'], [1.0]),
        'message_flows': (['label'], [1.0])
    }

    if component not in sub_components_weights:
        return 0  # If the component type doesn't match, return 0 similarity

    sub_components, weights = sub_components_weights[component]
    similarities = []
    existing_weights = 0  # To normalize only the used weights

    # Calculate weighted cosine similarity for each sub-component
    for sub_component, weight in zip(sub_components, weights):
        target_value = target_node.get(sub_component, None)
        other_value = other_node.get(sub_component, None)

        # Skip if either value is missing
        if target_value is None or other_value is None:
            continue

        # Apply cosine similarity for textual comparison
        similarity = cosine_similarity(str(target_value), str(other_value))
        similarities.append(weight * similarity)
        existing_weights += weight  # Only sum up existing weights

    # If no valid components were compared, return 0
    if existing_weights == 0:
        return 0

    # Normalize by the sum of existing weights
    return sum(similarities) / existing_weights


def calculate_structural_similarity(target_node_data, other_node_data):
    """
    Menghitung kemiripan struktural antara dua node menggunakan Graph Edit Distance (GED).
    - Cost Insert, Delete, Substitusi = 1
    - Jika tipe node berbeda → Cost 2 (karena harus delete + insert)
    - Jika target node missing → Cost 1 (karena hanya perlu insert)
    - Jika tipe sama tapi kontennya beda → Cost 1 (karena hanya perlu subtitusi)
    
    Hasil Similarity:
    - Cost 0 → Similarity = 1.0 (100%)
    - Cost 1 → Similarity = 0.5 (50%)
    - Cost 2 → Similarity = 0.0 (0%)
    """
    
    cost_insert = 1
    cost_delete = 1
    cost_substitute = 1
    
    # Ambil tipe dan konten dari kedua node
    target_type = target_node_data.get('type', 'Unknown')
    other_type = other_node_data.get('type', 'Unknown')
    
    target_label = target_node_data.get('label', 'Unknown')
    other_label = other_node_data.get('label', 'Unknown')

    # Jika salah satu node hilang
    if not target_node_data or not other_node_data:
        cost = cost_insert
    # Jika tipe berbeda, maka butuh delete + insert
    elif target_type != other_type:
        cost = cost_delete + cost_insert  # Total cost = 2
    # Jika tipe sama tetapi label berbeda, hanya perlu subtitusi
    elif target_label != other_label:
        cost = cost_substitute
    # Jika sama persis, tidak ada cost
    else:
        cost = 0

    # Konversi cost ke similarity
    if cost == 0:
        similarity = 1.0  # 100% similarity
    elif cost == 1:
        similarity = 0.5  # 50% similarity
    else:
        similarity = 0.0  # 0% similarity

    print(f"Cost: {cost}, Structural Similarity: {similarity}")
    
    return similarity, cost


def detect_missing_nodes_by_count(graph):
    """
    Detects missing nodes in the graph based on their count.
    A node is considered missing if it does not appear at least twice in the graph.
    """
    node_counts = {}
    
    # Count occurrences of each node in edges (both source and target)
    for source, target in graph.edges():
        node_counts[source] = node_counts.get(source, 0) + 1
        node_counts[target] = node_counts.get(target, 0) + 1

    # Identify nodes with less than 2 occurrences
    missing_nodes = [node for node, count in node_counts.items() if count < 2 and node not in ("StartEvent", "EndEvent")]
    # print("node_counts:", node_counts)
    # print("Missing Node:", missing_nodes)
    return missing_nodes


def calculate_combined_similarity_with_ged(target_graph, target_node, other_graph, other_node, component):
    """
    Menghitung kemiripan kombinasi (semantic + structural) antar node.
    - **Semantic similarity** dihitung dengan cosine similarity.
    - **Structural similarity** dihitung dengan Graph Edit Distance (GED).
    """

    # Ambil data node
    target_node_data = target_node[1]
    other_node_data = other_node[1]

    # Hitung kemiripan semantic
    semantic_similarity = calculate_semantic_similarity(target_node_data, other_node_data, component)

    # Hitung kemiripan structural menggunakan GED
    structural_similarity, cost = calculate_structural_similarity(target_node_data, other_node_data)
    
    print(f"Semantic Similarity: {semantic_similarity}, Structural Similarity: {structural_similarity}")

    # Kombinasikan dengan bobot yang sama (50% semantic, 50% structural)
    return semantic_similarity, structural_similarity, cost


def compare_bpmn_with_missing_node_detection(target_data, other_data_list):
    """
    Membandingkan target graph node-by-node dengan graph lain,
    menggunakan GED Greedy untuk structural similarity dan semantic similarity,
    serta mendeteksi missing nodes langsung dari target graph.
    """
    target_graph = convert_to_interstructural_with_order(target_data)
    target_nodes = list(target_graph.nodes(data=True))
    
    best_graph = None
    best_graph_name = None
    best_similarity = 0
    best_semantic = 0
    best_structural = 0
    best_total_cost = 0
    best_missing_nodes = []

    # Detect missing nodes from the target graph
    target_missing_nodes = detect_missing_nodes_by_count(target_graph)
    print("Target Missing Nodes:", target_missing_nodes)

    for other_data in other_data_list:
        current_missing_nodes = []
        print(f"\nComparing with: {other_data['file_name']}")
        other_graph = convert_to_interstructural_with_order(other_data)
        other_nodes = list(other_graph.nodes(data=True))

        print(f"{other_data['file_name']} Nodes:", list(other_graph.nodes()))

        total_semantic_similarity_graph = 0
        total_structural_similarity_graph = 0
        total_cost_graph = 0
        total_nodes_compared_graph = 0
        total_similarity = 0
        valid = True

        i, j = 0, 0
        while i < len(target_nodes) and j < len(other_nodes):
            target_nc = target_nodes[i]
            other_nc = other_nodes[j]

            component_type = 'activities' if target_nc[1].get('type') != 'Transition' else 'transitions'

            if target_nc[0] in target_missing_nodes:
                print(f"Missing node detected: {target_nc[0]} - Reason: Consecutive non-Transition nodes")

                match_found = False
                for k in range(j, len(other_nodes)):
                    other_nc_next = other_nodes[k]
                    semantic_similarity, structural_similarity, cost = calculate_combined_similarity_with_ged(
                        target_graph, target_nc, other_graph, other_nc_next, component_type
                    )
                    
                    total_cost_graph += cost

                    combined_similarity_next = 0.5 * semantic_similarity + 0.5 * structural_similarity

                    print(f"{target_nc[0]} vs {other_nc_next[0]}: {combined_similarity_next}")

                    if combined_similarity_next > 0.5:
                        print(f"Match {target_nc[0]} vs {other_nc_next[0]}: {combined_similarity_next}")
                        match_found = True
                        total_similarity += combined_similarity_next
                        total_nodes_compared_graph += 1

                        total_semantic_similarity_graph += semantic_similarity
                        total_structural_similarity_graph += structural_similarity

                        j = k  
                        break
                    else:
                        current_missing_nodes.append(other_nc_next[0])

                if not match_found:
                    print(f"Disqualifying graph: {other_data['file_name']} - Reason: Could not find match for missing node {target_nc[0]}")
                    valid = False
                    break
            
            else:
                semantic_similarity, structural_similarity, cost = calculate_combined_similarity_with_ged(
                    target_graph, target_nc, other_graph, other_nc, component_type
                )
                
                total_cost_graph += cost

                combined_similarity = 0.5 * semantic_similarity + 0.5 * structural_similarity

                print(f"Current: {target_nc[0]} vs {other_nc[0]} - Similarity: {combined_similarity}")

                if combined_similarity < 0.5:
                    print(f"Disqualifying graph: {other_data['file_name']} at node: {target_nc[0]} vs {other_nc[0]} - Similarity below threshold")
                    valid = False
                    break

                total_similarity += combined_similarity
                total_nodes_compared_graph += 1

                total_semantic_similarity_graph += semantic_similarity
                total_structural_similarity_graph += structural_similarity

            i += 1
            j += 1

        if valid and total_nodes_compared_graph > 0:
            avg_similarity = total_similarity / total_nodes_compared_graph
            avg_semantic_similarity_graph = (total_semantic_similarity_graph / total_nodes_compared_graph)
            avg_structural_similarity_graph = (total_structural_similarity_graph / total_nodes_compared_graph)

            print(f"\nGraph: {other_data['file_name']} - Average Combined Similarity: {avg_similarity:.2f}")
            print(f"Graph: {other_data['file_name']} - Total Semantic Similarity: {avg_semantic_similarity_graph}")
            print(f"Graph: {other_data['file_name']} - Total Structural Similarity: {avg_structural_similarity_graph}")
            print(f"Graph: {other_data['file_name']} - Total Cost: {total_cost_graph}")

            if avg_similarity > best_similarity:
                best_graph = other_graph
                best_graph_name = other_data.get('file_name', 'Unknown')
                best_similarity = avg_similarity
                best_semantic = avg_semantic_similarity_graph
                best_structural = avg_structural_similarity_graph
                best_missing_nodes = current_missing_nodes
                best_cost = total_cost_graph

    print(f"\nFinal Results: {target_data.get('file_name', 'Unknown')}")
    print("Best Graph:", best_graph_name)
    print("Best Similarity:", best_similarity)
    print("Best Semantic:", best_semantic)
    print("Best Structural:", best_structural)
    print("Best Cost:", best_cost)
    print(f"Target Missing Nodes: {target_missing_nodes}")
    print("Matching nodes:", best_missing_nodes)
    
    return best_graph, best_graph_name, target_graph



# Visualize graph with updated labels
def visualize_graph_with_labels(G, title="Graph Visualization", file_name="Graph.png"):
    # Update node labels to include "Start" and "End"
    # label_start_and_end_nodes(G)

    # Use the updated labels for visualization
    pos = nx.spring_layout(G)
    labels = nx.get_node_attributes(G, 'label')
    # Draw the graph with labels
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos, with_labels=True, labels=labels, 
            node_color='lightblue', edge_color='black', node_size=1000, font_size=12, font_weight='bold')
    
    # Set title and adjust layout
    plt.title(title, fontsize=14)
    # plt.tight_layout()
    plt.savefig(file_name)
    plt.show()


if __name__ == "__main__":
    folder_path = 'XPDL_3'
    target_file = 'XPDL_3/Sales Transactions.xpdl'

    # Parse target file
    target_data = parse_xpdl(target_file)

    # Parse other files
    other_data_list = []
    for file_name in os.listdir(folder_path):
        if (file_name.endswith(".xpdl") or file_name.endswith(".xml"))  and file_name != os.path.basename(target_file):
            file_path = os.path.join(folder_path, file_name)
            parsed_data = parse_xpdl(file_path)
            parsed_data['file_name'] = file_name  # Add file name
            other_data_list.append(parsed_data)
        else:
            target_data['file_name'] = file_name

    # Compare BPMNs
    best_graph, best_graph_name, target_graph = compare_bpmn_with_missing_node_detection(target_data, other_data_list)

    # Visualize the main graph
    visualize_graph_with_labels(target_graph, f"Graph Visualization of Target XPDL: {target_file}", f"Visual/Target.png")
    if best_graph:
        visualize_graph_with_labels(best_graph, f"Graph Visualization of Best Match: {best_graph_name}", f"Visual/Match.png")